In [1]:
import time
import json
import requests
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
def Domestic():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
    reponse = requests.get(url=url).json()
    data = json.loads(reponse['data'])
    return data


def Oversea():
    url = 'https://api.inews.qq.com/newsqa/v1/automation/modules/list?modules=FAutoCountryConfirmAdd,WomWorld,WomAboard'
    reponse = requests.post(url=url).json()
    data = reponse['data']
    return data


domestic = Domestic()
oversea = Oversea()

print(domestic.keys())
print(oversea.keys())

dict_keys(['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree'])
dict_keys(['FAutoCountryConfirmAdd', 'WomAboard', 'WomWorld'])


In [3]:
# 提取各地区数据明细
areaTree = domestic['areaTree']
# 查看并分析具体数据
areaTree

[{'name': '中国',
  'today': {'confirm': 165, 'isUpdated': True},
  'total': {'nowConfirm': 4268,
   'confirm': 130155,
   'suspect': 3,
   'dead': 5699,
   'deadRate': '4.38',
   'showRate': False,
   'heal': 120188,
   'healRate': '92.34',
   'showHeal': True,
   'wzz': 0},
  'children': [{'name': '台湾',
    'today': {'confirm': 27,
     'confirmCuts': 0,
     'isUpdated': True,
     'tip': '',
     'wzz_add': 0},
    'total': {'nowConfirm': 2261,
     'confirm': 16853,
     'suspect': 0,
     'dead': 850,
     'deadRate': '5.04',
     'showRate': False,
     'heal': 13742,
     'healRate': '81.54',
     'showHeal': True,
     'wzz': 0},
    'children': [{'name': '地区待确认',
      'today': {'confirm': 27, 'confirmCuts': 0, 'isUpdated': True},
      'total': {'nowConfirm': 2261,
       'confirm': 16853,
       'suspect': 0,
       'dead': 850,
       'deadRate': '5.04',
       'showRate': False,
       'heal': 13742,
       'healRate': '81.54',
       'showHeal': True,
       'wzz': 0}}]},


In [4]:
# 提取国外地区数据明细
foreignList = oversea['WomAboard']
# 查看并分析具体数据
foreignList

[{'continent': '北美洲',
  'name': '美国',
  'confirm': 52510978,
  'confirmAdd': 232383,
  'dead': 833029,
  'deadCompare': 1634,
  'heal': 40908146,
  'healCompare': 104606,
  'pub_date': '20211223',
  'y': '2021',
  'date': '12.23',
  'nowConfirm': 10769803,
  'nowConfirmCompare': 126143,
  'confirmAddCut': 0,
  'suspect': 0,
  'confirmCompare': 0},
 {'continent': '亚洲',
  'name': '印度',
  'confirm': 34765976,
  'confirmAdd': 7495,
  'dead': 478759,
  'deadCompare': 434,
  'heal': 34201966,
  'healCompare': 6906,
  'pub_date': '20211223',
  'y': '2021',
  'date': '12.23',
  'nowConfirm': 85251,
  'nowConfirmCompare': 155,
  'confirmAddCut': 0,
  'suspect': 0,
  'confirmCompare': 0},
 {'continent': '南美洲',
  'name': '巴西',
  'confirm': 22222928,
  'confirmAdd': 3451,
  'dead': 618128,
  'deadCompare': 137,
  'heal': 21414318,
  'healCompare': 0,
  'pub_date': '20211223',
  'y': '2021',
  'date': '12.23',
  'nowConfirm': 190482,
  'nowConfirmCompare': 3314,
  'confirmAddCut': 0,
  'suspect': 0

In [5]:
china_data = areaTree[0]['children'] 
china_list = []
for a in range(len(china_data)):
    province = china_data[a]['name']  
    confirm = china_data[a]['total']['confirm'] 
    heal = china_data[a]['total']['heal']  
    dead = china_data[a]['total']['dead']  
    nowConfirm = confirm - heal - dead 
    china_dict = {} 
    china_dict['province'] = province  
    china_dict['nowConfirm'] = nowConfirm 
    china_list.append(china_dict) 

china_data = pd.DataFrame(china_list)
#china_data.loc[china_data['province']=='广西', 'province'] = '广西壮族自治区'
#china_data.loc[china_data['province']=='西藏', 'province'] = '西藏自治区'
#china_data.loc[china_data['province']=='新疆', 'province'] = '新疆维吾尔自治区'
#china_data.loc[china_data['province']=='内蒙古', 'province'] = '内蒙古自治区'
china_data.to_excel("国内疫情.xlsx", index=False) #存储为EXCEL文件
china_data.head()

,province,nowConfirm
0,台湾,2261
1,浙江,512
2,内蒙古,334
3,陕西,259
4,广西,226


In [7]:
world_data = foreignList  
world_list = []

for a in range(len(world_data)):
    # 提取数据
    country = world_data[a]['name']
    nowConfirm = world_data[a]['nowConfirm']  
    confirm = world_data[a]['confirm']
    dead = world_data[a]['dead']  
    heal = world_data[a]['heal'] 
    # 存放数据
    world_dict = {}
    world_dict['country'] = country
    world_dict['nowConfirm'] = nowConfirm
    world_dict['confirm'] = confirm
    world_dict['dead'] = dead
    world_dict['heal'] = heal
    world_list.append(world_dict)

world_data = pd.DataFrame(world_list)
world_data.to_excel("国外疫情.xlsx", index=False)
world_data.head()

,country,nowConfirm,confirm,dead,heal
0,美国,10769803,52510978,833029,40908146
1,印度,85251,34765976,478759,34201966
2,巴西,190482,22222928,618128,21414318
3,英国,1577420,11647473,147573,9922480
4,俄罗斯,895193,10292983,300269,9097521


In [8]:
world_data.loc[world_data['country'] == "中国", :]

,country,nowConfirm,confirm,dead,heal


In [9]:
confirm = areaTree[0]['total']['confirm']  # 提取中国累计确诊数据
heal = areaTree[0]['total']['heal']  # 提取中国累计治愈数据
dead = areaTree[0]['total']['dead']  # 提取中国累计死亡数据
nowConfirm = confirm - heal - dead  # 计算中国现有确诊数量

world_data = world_data.append(
    {
        'country': "中国",
        'nowConfirm': nowConfirm,
        'confirm': confirm,
        'heal': heal,
        'dead': dead
    },
    ignore_index=True)

In [10]:
world_data.loc[world_data['country'] == "中国", :]

,country,nowConfirm,confirm,dead,heal
220,中国,4268,130155,5699,120188


In [11]:
import pyecharts.options as opts
from pyecharts.charts import Map
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [20]:
m = Map()
m.add("", [list(z) for z in zip(list(china_data["province"]), list(china_data["nowConfirm"]))], maptype="china", is_map_symbol_show=False)
m.set_global_opts(
    title_opts=opts.TitleOpts(title="COVID-19中国现有地区现有确诊人数地图"),
    visualmap_opts=opts.VisualMapOpts(
        is_piecewise=True,
        pieces=[
            {
                "min": 5000,
                "label": '>5000',
                "color": "#893448"
            },  # 不指定 max，表示 max 为无限大
            {
                "min": 1000,
                "max": 4999,
                "label": '1000-4999',
                "color": "#ff585e"
            },
            {
                "min": 500,
                "max": 999,
                "label": '500-1000',
                "color": "#fb8146"
            },
            {
                "min": 101,
                "max": 499,
                "label": '101-499',
                "color": "#ffA500"
            },
            {
                "min": 10,
                "max": 100,
                "label": '10-100',
                "color": "#ffb248"
            },
            {
                "min": 1,
                "max": 9,
                "label": '1-9',
                "color": "#fff2d1"
            },
            {
                "max": 1,
                "label": '0',
                "color": "#ffffff"
            }
        ]))

In [22]:
m.load_javascript()

In [23]:
m.render_notebook()

In [14]:
world_name = pd.read_excel("国家中英文对照表.xlsx")
world_data_t = pd.merge(world_data,
                        world_name,
                        left_on="country",
                        right_on="中文",
                        how="inner")

world_data_t

,country,nowConfirm,confirm,dead,heal,英文,中文
0,美国,10769803,52510978,833029,40908146,United States,美国
1,印度,85251,34765976,478759,34201966,India,印度
2,巴西,190482,22222928,618128,21414318,Brazil,巴西
3,英国,1577420,11647473,147573,9922480,United Kingdom,英国
4,俄罗斯,895193,10292983,300269,9097521,Russia,俄罗斯
...,...,...,...,...,...,...,...
221,瓦努阿图,0,7,1,6,Vanuatu,瓦努阿图
222,马绍尔群岛,0,4,0,4,Marshall Islands,马绍尔群岛
223,萨摩亚,0,3,0,3,Samoa,萨摩亚
224,密克罗尼西亚,0,1,0,1,Micronesia,密克罗尼西亚


In [18]:
m2 = Map()
m2.add("", [list(z) for z in zip(list(world_data_t["英文"]), list(world_data_t["nowConfirm"]))], maptype="world", is_map_symbol_show=False)
m2.set_global_opts(title_opts=opts.TitleOpts(title="COVID-19世界各国现有确诊人数地图"),
                   visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                                                     pieces=[{
                                                         "min": 10000000,
                                                         "label": '≥1000万',
                                                         "color": '#880000'
                                                     }, {
                                                         "min": 5000000,
                                                         "max": 9999999,
                                                         "label": '500万-1000万',
                                                         "color": '#a42d00'
                                                     }, {
                                                         "min": 1000000,
                                                         "max": 4999999,
                                                         "label": '100万-500万',
                                                         "color": "#c63300"
                                                     }, {
                                                         "min": 100000,
                                                         "max": 999999,
                                                         "label": '10万-100万',
                                                         "color": "#e63f00"
                                                     }, {
                                                         "min": 10000,
                                                         "max": 99999,
                                                         "label": '1万-10万',
                                                         "color": "#ff5511"
                                                     }, {
                                                         "min": 5000,
                                                         "max": 9999,
                                                         "label": '5000-9999',
                                                         "color": "#ff7744"
                                                     }, {
                                                         "min": 10,
                                                         "max": 4999,
                                                         "label": '10-4999',
                                                         "color": "#ffa488"
                                                     }, {
                                                         "min": 0,
                                                         "max": 9,
                                                         "label": '0-9',
                                                         "color": "#fff2d1"
                                                     }]))
"""取消显示国家名称"""
m2.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

In [19]:
m2.load_javascript()
m2.render_notebook()